In [1]:
import sys
import os
sys.path.append('/Users/masahiro/song_recommender/music_recommender/')

In [2]:
#import featurization as feat
import k_means
from testing import evaluate, evaluate_fuzzy_model

In [3]:
import numpy as np 
import pandas as pd 
from math import floor
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
from sklearn.preprocessing import MinMaxScaler
import statistics as st
import skfuzzy as fuzz
import os
import pickle

In [7]:
root_path = os.getcwd()[:-5]
filepath = f"{root_path}/playlist_data/sampledata/"
filename = 'mpd.slice.0-999.json'
fpath_name = f"{filepath}{filename}"
with open(fpath_name) as data_file:    
    data = json.load(data_file)

playlists_df = pd.json_normalize(data['playlists'])
    
filename = 'song.slice.0-49999.csv'
fpath_name = f"{filepath}{filename}"

songs_df = pd.read_csv(fpath_name)


In [5]:
filename = 'model_energy.pkl'

model = pickle.load(open(filename, 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'model_energy.pkl'

In [8]:
playlists = playlists_df[:100]
songs_df = songs_df.iloc[:,2:]
features = songs_df.select_dtypes(include=np.number).columns

In [11]:
evaluate_fuzzy_model(features, playlists, n_suggestions=1)

{'artist_match_rate': 0.0, 'track_match_rate': 0.0}

In [15]:
playlists_df.tracks[0][0]

{'pos': 0,
 'artist_name': 'Missy Elliott',
 'track_uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI',
 'artist_uri': 'spotify:artist:2wIVse2owClT7go1WT98tk',
 'track_name': 'Lose Control (feat. Ciara & Fat Man Scoop)',
 'album_uri': 'spotify:album:6vV5UrXcfyQD1wu4Qo2I9K',
 'duration_ms': 226863,
 'album_name': 'The Cookbook'}

In [14]:
songs_df

,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,num_playlist_appearances,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,time_signature
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226864,The Cookbook,6840,0.904,0.813,...,0.0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,4.0
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,13204,0.774,0.838,...,0.0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,audio_features,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,4.0
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),16913,0.664,0.758,...,0.0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,audio_features,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,4.0
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267267,Justified,9065,0.892,0.714,...,0.0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,audio_features,spotify:track:1AWQoqb9bSvzTjaLralEkT,4.0
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,23172,0.853,0.606,...,1.0,0.0713,0.05610,0.000000,0.3130,0.654,94.759,audio_features,spotify:track:1lzr43nnXAijIGYnCT8M8H,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,Headhunterz,spotify:track:5QBQ6uA3sQFBSJJt8w9iat,spotify:artist:6C0KWmCdqrLU2LzzWBPbOy,Lift Me Up,spotify:album:12oB47jQouCO3smnZHrjiT,179488,Lift Me Up,70,0.409,0.610,...,0.0,0.0447,0.06960,0.000006,0.0655,0.468,101.576,audio_features,spotify:track:5QBQ6uA3sQFBSJJt8w9iat,5.0
49996,Keanu Silva,spotify:track:52DhbZEJ6lPZqJSrOPqSKo,spotify:artist:1zLMhO4zzzxt5PMV4wMS3y,Children - RMFB Remix Edit,spotify:album:6fLFnjmVbaTIhoQUpstUTY,195000,Children,23,0.222,0.814,...,0.0,0.1010,0.00226,0.902000,0.2610,0.242,153.407,audio_features,spotify:track:52DhbZEJ6lPZqJSrOPqSKo,4.0
49997,Myon,spotify:track:7dCmQbhItGA7JgdBUgY18J,spotify:artist:0nTbVTXLLbBA4xCtn0cFkv,Round We Go - Radio Edit,spotify:album:1FhkQ24VBtdglGRHJbQlcL,251250,Round We Go,44,0.490,0.905,...,0.0,0.0717,0.03950,0.000059,0.6860,0.210,128.016,audio_features,spotify:track:7dCmQbhItGA7JgdBUgY18J,4.0
49998,Alison Wonderland,spotify:track:28QlLenBGFRjxMmKONi9x2,spotify:artist:11gWrKZMBsGQWmobv3oNfW,U Don't Know - Vincent Remix,spotify:album:6ahNh15MTKTZb2IQJMnqBy,217795,Run,156,0.414,0.767,...,0.0,0.0650,0.00515,0.000000,0.0996,0.210,137.945,audio_features,spotify:track:28QlLenBGFRjxMmKONi9x2,4.0


In [10]:
songs_df.columns

Index(['artist_name', 'track_uri', 'artist_uri', 'track_name', 'album_uri',
       'duration_ms', 'album_name', 'num_playlist_appearances', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'uri',
       'time_signature'],
      dtype='object')

In [11]:
songs = songs_df.select_dtypes(include=np.number)

In [6]:
# Create partial playlists to test the model on

partial_playlist = playlists_df.tracks.apply(lambda x: x[:max(1,floor(len(x)*(1-0.5)))])
partial_playlist = partial_playlist[:200].to_frame()
partial_playlist['pid'] = range(200)
partial_playlist

,tracks,pid
0,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",0
1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",1
2,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",2
3,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",3
4,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",4
...,...,...
195,"[{'pos': 0, 'artist_name': 'Dara Maclean', 'tr...",195
196,"[{'pos': 0, 'artist_name': 'The Avett Brothers...",196
197,"[{'pos': 0, 'artist_name': 'ODESZA', 'track_ur...",197
198,"[{'pos': 0, 'artist_name': 'Gwen McCrae', 'tra...",198


In [26]:
n_suggestions = 5

In [10]:
# Evaluation of K-means using song features only

# Normalize features
songs = songs_df.select_dtypes(include=np.number).to_numpy()
minmax = MinMaxScaler()
songs = minmax.fit_transform(songs)

# Fit the model
#model = k_means.kmeans_fit(songs, N_clusters=50)



In [200]:
def k_means_rec(playlists):
    
    recs = {'pid':[], 'tracks':[]}
    
    for i in range(len(playlists)):
        
        recs['pid'].append(playlists.pid[i])
                           
        playlist = playlists.tracks[i]
        uri_list = [track['track_uri'] for track in playlist]
        
        playlist = songs[[uri in uri_list for uri in song_df['uri']]]
        
        # Predict the cluster
        clusters = model.predict(playlist)
        cluster = st.mode(clusters)

        # Suggest songs from the cluster
        
        recs['tracks'].append(song_df[model.labels_ == cluster].sample(n_suggestions).to_dict(orient='records'))
        
    return pd.DataFrame(recs)

def k_means_rec_artist_priority(playlists):
    
    recs = {'pid':[], 'tracks':[]}
    
    for i in range(len(playlists)):
        
        recs['pid'].append(playlists.pid[i])
                           
        playlist = playlists.tracks[i]
        uri_list = [track['track_uri'] for track in playlist]
        
        playlist = songs[[uri in uri_list for uri in song_df['uri']]]
        playlist_df = song_df[[uri in uri_list for uri in song_df['uri']]]
        
        artists = playlist_df.artist_name.values

        
        # Predict the cluster
        clusters = model.predict(playlist)
        cluster = st.mode(clusters)

        # Suggest songs from the cluster
        
        crit1 = song_df[[artist in artists for artist in song_df['artist_name'].values]]
        crit2 = song_df[model.labels_ == cluster]
        
        crit = pd.merge(crit1, crit2, how='inner', on='track_uri', suffixes=('', '_y'))
        n = len(crit)
        
        recs['tracks'].append(crit.sample(min(n,n_suggestions)).to_dict(orient='records'))
        
    return pd.DataFrame(recs)


def baseline_rec(playlists):
    
    recs = {'pid':[], 'tracks':[]}
    
    for i in range(len(playlists)):
        
        recs['pid'].append(playlists.pid[i])
                           
        playlist = playlists.tracks[i]
        uri_list = [track['track_uri'] for track in playlist]
        
        playlist = song_df[[uri in uri_list for uri in song_df['uri']]]
        
        artists = playlist.artist_name.values

        # Suggest songs with the same artist
        
        recs['tracks'].append(song_df[[artist in artists for artist in song_df['artist_name'].values]].sample(n_suggestions, replace=True).to_dict(orient='records'))
        
    return pd.DataFrame(recs)

In [201]:
recs = k_means_rec_artist_priority(partial_playlist)

In [207]:
recs_b = baseline_rec(partial_playlist)

In [204]:
evaluate(recs)

{'artist_match_rate': 0.14291666666666664,
 'track_match_rate': 0.013166666666666667}

In [208]:
evaluate(recs_b)

{'artist_match_rate': 0.13516666666666657, 'track_match_rate': 0.012}

In [43]:
 a = pd.Series([{'a':2},2,3])

In [44]:
a

0    {'a': 2}
1           2
2           3
dtype: object

In [46]:
a[0]

{'a': 2}

In [11]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# Step 1: Load the song data
song_data = songs_df.select_dtypes(include=np.number)

# Step 2: Define the input songs
#input_songs = ['White Christmas', 'Let It Snow! Let It Snow! Let It Snow!', 'Jingle Bell Rock', 'Winter Wonderland']

# Step 4: Preprocess numerical features
# numerical_features = ['danceability', 'energy', 'key', 'loudness',
#        'speechiness', 'acousticness', 'instrumentalness', 'liveness',
#        'valence', 'tempo', 'song_popularity', 'artist_popularity', 'album_popularity'] #['artist_fraction', 'playlist_description_encoded']  # Replace with the actual numerical feature column names
all_songs_numerical = song_data
input_songs_numerical = song_data

# Step 5: Preprocess categorical features
#categorical_features = ['track_track_name', 'playlist_name']  # Replace with the actual categorical feature column names
#all_songs_categorical = song_data[categorical_features]
#input_songs_categorical = input_songs_data[categorical_features]

# Step 6: Perform one-hot encoding on categorical features
encoder = OneHotEncoder(sparse_output=False)
#all_songs_categorical_encoded = encoder.fit_transform(all_songs_categorical)
#input_songs_categorical_encoded = encoder.transform(input_songs_categorical)

# Step 7: Combine numerical and categorical features
all_songs_combined = all_songs_numerical #np.concatenate((all_songs_numerical, all_songs_categorical_encoded), axis=1)
input_songs_combined = input_songs_numerical #np.concatenate((input_songs_numerical, input_songs_categorical_encoded), axis=1)

# Step 8: Scale the feature values
scaler = MinMaxScaler()
X_combined = scaler.fit_transform(all_songs_combined)
input_songs_combined_scaled = scaler.transform(input_songs_combined)

# Step 9: Train the fuzzy clustering model
num_clusters = 7
cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(X_combined.T, num_clusters, m=1.00001, error=0.000005, maxiter=50000)

# Step 10: Predict the degree of membership for all songs
all_songs_membership = fuzz.cluster.cmeans_predict(X_combined.T, cntr, 2, error=0.000005, maxiter=50000)[0]

# Step 11: Predict the degree of membership for the input songs
input_songs_membership = fuzz.cluster.cmeans_predict(input_songs_combined_scaled.T, cntr, 2, error=0.000005, maxiter=50000)[0]


In [12]:
input_songs_membership

array([[0.08654479, 0.09750568, 0.07947996, ..., 0.08992908, 0.0658463 ,
        0.06580303],
       [0.26326176, 0.2938084 , 0.1941167 , ..., 0.16461758, 0.1694723 ,
        0.15511537],
       [0.07985084, 0.08244089, 0.09008581, ..., 0.13376431, 0.09271508,
        0.09235441],
       ...,
       [0.36985724, 0.3135498 , 0.43036416, ..., 0.36505072, 0.50433105,
        0.51400599],
       [0.0429733 , 0.04618618, 0.04591198, ..., 0.06364876, 0.04409037,
        0.04118924],
       [0.05532144, 0.05896668, 0.05763012, ..., 0.07529766, 0.05245874,
        0.05043179]])

In [16]:

i = 0
cluster_num_string_array = []
sorted_indices_array = []

for input_membership_array in input_songs_membership.T:
    sorted_indices = np.argsort(input_membership_array)[::-1]
    sorted_indices_array.append(sorted_indices)
    
    cluster_num_string = "Cluster ranks for Input Track {}:    ".format(i)
    cluster_num_string_array.append(cluster_num_string)
    
    i=i+1
    
test_encoding = tuple(zip(cluster_num_string_array , sorted_indices_array))

for cluster_rankings in test_encoding:
    cluster_label = cluster_rankings[0]
    cluster_array = cluster_rankings[1].tolist()


In [22]:
test_encoding[:10]

(('Cluster ranks for Input Track 0:    ', array([4, 1, 3, 0, 2, 6, 5])),
 ('Cluster ranks for Input Track 1:    ', array([4, 1, 3, 0, 2, 6, 5])),
 ('Cluster ranks for Input Track 2:    ', array([4, 1, 3, 2, 0, 6, 5])),
 ('Cluster ranks for Input Track 3:    ', array([4, 1, 3, 0, 2, 6, 5])),
 ('Cluster ranks for Input Track 4:    ', array([3, 2, 0, 6, 4, 1, 5])),
 ('Cluster ranks for Input Track 5:    ', array([3, 2, 0, 6, 4, 1, 5])),
 ('Cluster ranks for Input Track 6:    ', array([3, 0, 2, 6, 4, 1, 5])),
 ('Cluster ranks for Input Track 7:    ', array([2, 3, 0, 6, 4, 1, 5])),
 ('Cluster ranks for Input Track 8:    ', array([4, 1, 3, 0, 2, 6, 5])),
 ('Cluster ranks for Input Track 9:    ', array([4, 1, 3, 0, 2, 6, 5])))